In [1]:
import numpy as np
import pandas as pd

In [2]:
import json

with open('model_df_weiwei_clean.json', 'r', encoding='UTF-8') as f:
    WEIWEI = json.load(f)
COL = ['index', 'name', 'code', 'time', 'price', 'time_1', 'price_1', 'price_dif_1', 'sell_1', 'buy_1', 'volume_1', 'variation_1', 'post_num_1', 'unique_id_1', 'click_1', 'like_1', 'dislike_1', 'time_2', 'price_2', 'price_dif_2', 'sell_2', 'buy_2', 'volume_2', 'variation_2', 'post_num_2', 'unique_id_2', 'click_2', 'like_2', 'dislike_2', 'time_3', 'price_3', 'price_dif_3', 'sell_3', 'buy_3', 'volume_3', 'variation_3', 'post_num_3', 'unique_id_3', 'click_3', 'like_3', 'dislike_3', 'mkt_cap', 'kospi', 'kosdaq', 'trash', 'yesterday_closing_price', 'is_maximum', 'is_minimum', 'price_volatility', 'price_trend', 'average_price_volatility', 'sell_minus_buy_1', 'sell_minus_buy_2', 'sell_minus_buy_3', 'is_price_gap_stable', 'price_gap_volatility', 'is_like_higher', 'volume_trend', 'post_num_trend', 'unique_id_trend', 'click_trend', 'price_increase', 'did_price_increase', 'did_price_033', 'did_price_100', 'did_price_150', 'kospi_ind', 'kosdaq_ind', 'time_slot', 'ko_inter', 'early_mor', 'morning', 'lunch', 'afternoon', 'late', 'mkt_change', 'alpha', 'per_now', 'kospi_1', 'kospi_2', 'kospi_3', 'kospi_answer', 'kosdaq_1', 'kosdaq_2', 'kosdaq_3', 'kosdaq_answer', 'kospi_trend', 'kosdaq_trend', 'kospi_increase', 'kosdaq_increase', 'market_increase', 'did_opening_price_increase', 'price_1_sq', 'price_dif_1_sq', 'sell_1_sq', 'buy_1_sq', 'volume_1_sq', 'variation_1_sq', 'post_num_1_sq', 'unique_id_1_sq', 'click_1_sq', 'like_1_sq', 'dislike_1_sq', 'price_2_sq', 'price_dif_2_sq', 'sell_2_sq', 'buy_2_sq', 'volume_2_sq', 'variation_2_sq', 'post_num_2_sq', 'unique_id_2_sq', 'click_2_sq', 'like_2_sq', 'dislike_2_sq', 'price_3_sq', 'price_dif_3_sq', 'sell_3_sq', 'buy_3_sq', 'volume_3_sq', 'variation_3_sq', 'post_num_3_sq', 'unique_id_3_sq', 'click_3_sq', 'like_3_sq', 'dislike_3_sq', 'mkt_cap_sq', 'yesterday_closing_price_sq', 'price_volatility_sq', 'price_trend_sq', 'average_price_volatility_sq', 'sell_minus_buy_1_sq', 'sell_minus_buy_2_sq', 'sell_minus_buy_3_sq', 'price_gap_volatility_sq', 'volume_trend_sq', 'post_num_trend_sq', 'unique_id_trend_sq', 'click_trend_sq', 'kospi_ind_sq', 'kosdaq_ind_sq', 'time_slot_sq', 'ko_inter_sq', 'mkt_change_sq', 'alpha_sq', 'per_now_sq', 'kospi_1_sq', 'kospi_2_sq', 'kospi_3_sq', 'kosdaq_1_sq', 'kosdaq_2_sq', 'kosdaq_3_sq', 'kospi_trend_sq', 'kosdaq_trend_sq', 'KNN', 'PLS', 'Logistic', 'Random Forest','Bagging']
df = pd.DataFrame(WEIWEI, columns=COL)

In [3]:
C = list(df.columns)
print(C)

['index', 'name', 'code', 'time', 'price', 'time_1', 'price_1', 'price_dif_1', 'sell_1', 'buy_1', 'volume_1', 'variation_1', 'post_num_1', 'unique_id_1', 'click_1', 'like_1', 'dislike_1', 'time_2', 'price_2', 'price_dif_2', 'sell_2', 'buy_2', 'volume_2', 'variation_2', 'post_num_2', 'unique_id_2', 'click_2', 'like_2', 'dislike_2', 'time_3', 'price_3', 'price_dif_3', 'sell_3', 'buy_3', 'volume_3', 'variation_3', 'post_num_3', 'unique_id_3', 'click_3', 'like_3', 'dislike_3', 'mkt_cap', 'kospi', 'kosdaq', 'trash', 'yesterday_closing_price', 'is_maximum', 'is_minimum', 'price_volatility', 'price_trend', 'average_price_volatility', 'sell_minus_buy_1', 'sell_minus_buy_2', 'sell_minus_buy_3', 'is_price_gap_stable', 'price_gap_volatility', 'is_like_higher', 'volume_trend', 'post_num_trend', 'unique_id_trend', 'click_trend', 'price_increase', 'did_price_increase', 'did_price_033', 'did_price_100', 'did_price_150', 'kospi_ind', 'kosdaq_ind', 'time_slot', 'ko_inter', 'early_mor', 'morning', 'lunc

In [4]:
gb = pd.DataFrame(df.groupby("time")[["kospi_answer", "kosdaq_answer"]].mean())

In [5]:
gb = gb[93:]

In [6]:
def simulation(VAR):
    
    gb[VAR + "_increase"] = 1 + (df[df[VAR] == 1].groupby("time")["price_increase"].mean())/100
    gb[VAR + "_balance"] = np.nan
    gb[VAR + "_balance"].loc["2018-02-22 10:30"] = 100

    c = 0
    for t, row in gb.iterrows():
        if c > 0:
            if np.isnan(row[VAR + "_increase"]):
                row[VAR + "_balance"] = gb[VAR + "_balance"].iloc[c-1]

            else:

                buying_amount = gb[VAR + "_balance"].iloc[c-1]
                buying_fee = buying_amount * 0.00015

                selling_amount = buying_amount * row[VAR + "_increase"]
                selling_fee = selling_amount * 0.00315

                balance = selling_amount - buying_fee - selling_fee

                row[VAR + "_balance"] = balance
        c += 1

In [7]:
method = ['KNN', 'PLS', 'Logistic', 'Random Forest', 'Bagging']

for m in method:
    simulation(m)

In [8]:
gb["KOSPI_balance"] = 100 * gb["kospi_answer"] / gb["kospi_answer"].iloc[0]
gb["KOSDAQ_balance"] = 100 * gb["kosdaq_answer"] / gb["kosdaq_answer"].iloc[0]

gb["KOSPI_balance"].iloc[len(gb) - 1] = gb["KOSPI_balance"].iloc[len(gb) - 1] * (1-0.00315) - 0.015
gb["KOSDAQ_balance"].iloc[len(gb) - 1] = gb["KOSDAQ_balance"].iloc[len(gb) - 1] * (1-0.00315) - 0.015

In [9]:
result = gb[["KOSPI_balance", "KOSDAQ_balance", "KNN_balance", "PLS_balance",\
             "Logistic_balance", "Random Forest_balance", "Bagging_balance"]]
result = result.reset_index()

In [10]:
result

,time,KOSPI_balance,KOSDAQ_balance,KNN_balance,PLS_balance,Logistic_balance,Random Forest_balance,Bagging_balance
0,2018-02-22 10:30,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
1,2018-02-22 10:40,100.055914,100.057478,100.000000,100.000000,99.435723,98.223523,99.224978
2,2018-02-22 10:50,99.970594,99.941373,100.000000,100.000000,102.912671,98.223523,99.224978
3,2018-02-22 11:00,100.133364,99.939073,100.000000,100.000000,102.912671,98.338462,99.224978
4,2018-02-22 11:10,100.200046,100.121853,100.000000,100.000000,102.912671,98.283255,99.224978
5,2018-02-22 11:20,100.167741,100.090815,100.000000,100.000000,102.912671,97.385316,99.224978
6,2018-02-22 11:30,100.066682,100.019542,99.493249,100.000000,103.131761,97.574522,99.224978
7,2018-02-22 11:40,100.068339,100.005748,99.340151,100.000000,103.269166,97.574522,99.224978
8,2018-02-22 11:50,100.100644,100.081619,99.049637,100.000000,103.631526,97.208130,99.224978
9,2018-02-22 12:00,100.095260,100.083918,99.049637,100.000000,103.631526,96.889292,98.205840


In [11]:
lst = list(result.columns)
print(lst)

['time', 'KOSPI_balance', 'KOSDAQ_balance', 'KNN_balance', 'PLS_balance', 'Logistic_balance', 'Random Forest_balance', 'Bagging_balance']


In [12]:
result.to_json('groupby_balance.json', orient='values')